In [1]:
import arkouda as ak
import arachne as ar
import pandas as pd
import time as time
import networkx as nx
import random


    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2023.11.15


In [2]:
from dotmotif import Motif, GrandIsoExecutor 

In [3]:
ak.connect("n51", 5555)

connected to arkouda server tcp://*:5555


In [5]:


# Define the file path to your graph dataset
file_path = '/scratch/users/md724/DataSets/web-BerkStan/web-BerkStan.txt'

# Load the dataset into a DataFrame, skipping the initial comments
df = pd.read_csv(file_path, delim_whitespace=True, comment='#', header=None, names=['FromNodeId', 'ToNodeId'])

df.rename(columns={'FromNodeId': 'src', 'ToNodeId': 'dst'}, inplace=True)

df['type'] = 'T1'

# Display the first few rows of the DataFrame to verify it loaded correctly
print(df.head())
# Concatenate 'src' and 'dst' columns and find unique elements for nodes
unique_nodes = pd.concat([df['src'], df['dst']]).unique()

# Number of unique nodes
num_unique_nodes = len(unique_nodes)

# Number of edges is the length of the DataFrame
num_edges = len(df)
d = num_edges/(num_unique_nodes*(num_unique_nodes -1))
print(f"Graph Density: {d}")

print(f"Number of unique nodes: {num_unique_nodes}")
print(f"Number of edges: {num_edges}")

   src  dst type
0    1    2   T1
1    1    3   T1
2    1    4   T1
3    1    5   T1
4    1    6   T1
Graph Density: 1.2413817875891928e-05
Number of unique nodes: 3072441
Number of edges: 117185083


In [4]:
microns = pd.read_csv("/scratch/users/oaa9/experimentation/data/connectome/MICrONS/soma_subgraph_synapses_spines_v185.csv")
microns


,id,pre_root_id,post_root_id,cleft_vx,spine_vol_um3,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
0,1484,648518346349539437,648518346349531254,798,0.133004,365476,231192,63280,91332,57836,1584,91369,57798,1582,91332,57748,1584
1,3056393,648518346349537978,648518346349531254,209,0.087794,312120,209816,9960,78050,52470,248,78030,52454,249,77980,52444,249
2,310116,648518346349538410,648518346349531254,869,0.234537,313596,185764,23920,78314,46428,598,78399,46441,598,78404,46410,598
3,1533059,648518346349538410,648518346349531254,231,0.081921,289560,174904,33920,72340,43756,849,72390,43726,848,72354,43704,849
4,2505779,648518346349538410,648518346349531254,669,0.105706,296056,162304,9520,73966,40608,231,74014,40576,238,73918,40562,232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,3267950,648518346349492682,648518346349537243,74,0.025112,452184,241916,37960,113060,60494,948,113046,60479,949,113026,60468,950
1957,1416588,648518346349536680,648518346349537243,211,0.029907,400144,257752,40280,99994,64410,1008,100036,64438,1007,100066,64456,1009
1958,801412,648518346349538527,648518346349537243,134,0.060255,399560,249432,37480,99860,62324,938,99890,62358,937,99906,62378,938
1959,402767,648518346349538711,648518346349531851,95,0.014755,395024,236312,76000,98768,59108,1902,98756,59078,1900,98734,59056,1901


In [5]:
microns.columns

Index(['id', 'pre_root_id', 'post_root_id', 'cleft_vx', 'spine_vol_um3',
       'ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm', 'pre_pos_x_vx',
       'pre_pos_y_vx', 'pre_pos_z_vx', 'ctr_pos_x_vx', 'ctr_pos_y_vx',
       'ctr_pos_z_vx', 'post_pos_x_vx', 'post_pos_y_vx', 'post_pos_z_vx'],
      dtype='object')

In [6]:
hemibrain_traced_roi_connections = pd.read_csv("/scratch/users/oaa9/experimentation/data/connectome/hemibrain/exported-traced-adjacencies-v1.2/traced-roi-connections.csv")
hemibrain_traced_roi_connections

,bodyId_pre,bodyId_post,roi,weight
0,200326126,264083994,SLP(R),3
1,200326126,295816140,ICL(R),2
2,200326126,295816140,SCL(R),2
3,200326126,295816140,PLP(R),1
4,200326126,296203440,SCL(R),1
...,...,...,...,...
4259619,7112622236,328283521,SMP(R),1
4259620,7112622236,357932060,SMP(R),1
4259621,7112622236,357940977,SMP(R),1
4259622,7112622236,358631450,SMP(R),1


In [7]:
c_elegans = pd.read_csv("/scratch/users/oaa9/experimentation/data/connectome/c.elegans/celegans_actual.csv")
print(c_elegans)
c_elegans.columns = c_elegans.columns.str.replace(" ", "")
c_elegans.columns = c_elegans.columns.str.strip()
c_elegans["post1"] = c_elegans["post1"].astype(str)
c_elegans["post2"] = c_elegans["post2"].astype(str)
c_elegans["post3"] = c_elegans["post3"].astype(str)
c_elegans["post4"] = c_elegans["post4"].astype(str)
#c_elegans

      continNum  EM series  pre    post       type      sec-tions     \
0            653       JSE   PVNL  PQR,hyp    chemical             1   
1            651       JSE     g0     PVCR    chemical             1   
2            652       JSE     g0     PVCR    chemical             1   
3            650       JSE     gx      r30    chemical             1   
4           2097       JSE     gx      r30    chemical             1   
...          ...       ...    ...      ...         ...           ...   
9668         999       N2W   e3VL     e2DL  electrical             1   
9669        1000       N2W  pm5VL      M2L  electrical             1   
9670        1064       N2W   g1AL       M4  electrical             2   
9671        1098       N2W    pm1      e3D  electrical             3   
9672        1153       N2W     M5       I5  electrical             2   

      part-ner Num-ber post1      post2    post3  post4     
0                    2        PQR      hyp    NaN      NaN  
1            

In [8]:
temp_cols = list(c_elegans.columns)
temp_cols.remove("post1")
temp_cols.remove("post2")
temp_cols.remove("post3")
temp_cols.remove("post4")
temp = {k:[] for k in temp_cols}
for index,row in c_elegans.iterrows():
    if row["post1"] != "nan":
        for k in temp_cols:
            if k == "post":
                temp[k].append(row["post1"])
            else:
                temp[k].append(row[k])
    if row["post2"] != "nan":
        for k in temp_cols:
            if k == "post":
                temp[k].append(row["post2"])
            else:
                temp[k].append(row[k])
    if row["post3"] != "nan":
        for k in temp_cols:
            if k == "post":
                temp[k].append(row["post3"])
            else:
                temp[k].append(row[k])
    if row["post4"] != "nan":
        for k in temp_cols:
            if k == "post":
                temp[k].append(row["post4"])
            else:
                temp[k].append(row[k])

In [9]:
c_elegans_from_dict = pd.DataFrame.from_dict(temp)
c_elegans_from_dict

,continNum,EMseries,pre,post,type,sec-tions,part-nerNum-ber
0,653,JSE,PVNL,PQR,chemical,1,2
1,653,JSE,PVNL,hyp,chemical,1,2
2,651,JSE,g0,PVCR,chemical,1,1
3,652,JSE,g0,PVCR,chemical,1,1
4,650,JSE,gx,r30,chemical,1,1
...,...,...,...,...,...,...,...
15637,999,N2W,e3VL,e3VL,electrical,1,1
15638,1000,N2W,pm5VL,pm5VL,electrical,1,1
15639,1064,N2W,g1AL,M4,electrical,2,1
15640,1098,N2W,pm1,e3D,electrical,3,1


In [10]:
neuron_dfs_in_pandas = [c_elegans_from_dict, microns, hemibrain_traced_roi_connections]


In [11]:
neuron_dfs_in_arkouda = [ak.DataFrame(pd_df) for pd_df in neuron_dfs_in_pandas]

In [12]:
ak_celegans = neuron_dfs_in_arkouda[0]
ak_microns = neuron_dfs_in_arkouda[1]
ak_hemibrain_traced_roi_connections = neuron_dfs_in_arkouda[2]

In [13]:
ak_celegans_gb = ak_celegans.groupby(["pre", "post"])
print(ak_celegans_gb)


In [14]:
ak_celegans_sorted = ak_celegans[ak_celegans_gb.permutation[ak_celegans_gb.segments]]
chemical_synapses = ak_celegans_sorted["type"] == "chemical"
ak_celegans_sorted = ak_celegans_sorted[chemical_synapses]

In [15]:
ak_celegans_nodes = ak.concatenate([ak_celegans_sorted["pre"], ak_celegans_sorted["post"]])
gb_celegans_nodes = ak.GroupBy(ak_celegans_nodes)
new_vertex_range = ak.arange(gb_celegans_nodes.unique_keys.size)
all_vertices = gb_celegans_nodes.broadcast(new_vertex_range)
ak_celegans_sorted["pre"] = all_vertices[0:ak_celegans_sorted.shape[0]]
ak_celegans_sorted["post"] = all_vertices[ak_celegans_sorted.shape[0]:]
ak_celegans_sorted

,continNum,EMseries,pre,post,type,sec-tions,part-nerNum-ber
175,548,JSE,234,341,chemical,1,2
1436,524,N2U,413,199,chemical,3,2
1001,2248,JSE,441,212,chemical,1,2
8160,5392,N2U,275,128,chemical,1,2
2845,1351,N2U,145,299,chemical,1,2
...,...,...,...,...,...,...,...
1438,525,N2U,319,273,chemical,1,2
10497,8018,N2U,328,270,chemical,1,2
4405,2432,N2U,147,128,chemical,1,2
5236,3095,N2U,84,396,chemical,1,2


In [16]:
ak_microns_gb = ak_microns.groupby(["pre_root_id", "post_root_id"])
ak_microns_sorted = ak_microns[ak_microns_gb.permutation[ak_microns_gb.segments]]
ak_microns_sorted

,id,pre_root_id,post_root_id,cleft_vx,spine_vol_um3,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
914,833178,648518346349492682,648518346349532066,137,0.038308,422536,270840,63160,105598,67710,1580,105634,67710,1579,105664,67714,1578
1956,3267950,648518346349492682,648518346349537243,74,0.025112,452184,241916,37960,113060,60494,948,113046,60479,949,113026,60468,950
1800,3271025,648518346349492682,648518346349537515,1658,0.299525,434544,283572,59680,108612,70826,1494,108636,70893,1492,108542,70874,1494
1955,1857741,648518346349492682,648518346349537609,300,0.043498,408280,263208,66280,102056,65756,1657,102070,65802,1657,102038,65832,1657
1353,3313914,648518346349492682,648518346349537790,153,0.048934,382272,243972,70920,95526,60946,1776,95568,60993,1773,95540,61002,1775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522,183324,648518346349540057,648518346349538426,76,0.016554,435336,264800,62000,108856,66194,1551,108834,66200,1550,108820,66214,1551
760,2147508,648518346349540057,648518346349538711,399,0.044378,410360,165008,49000,102526,41256,1224,102590,41252,1225,102618,41240,1224
1787,2873830,648518346349540057,648518346349539832,283,0.109871,451480,198912,86120,112786,49686,2154,112870,49728,2153,112756,49766,2155
666,3323352,648518346349540057,648518346349539890,891,0.134686,343440,240984,28080,85758,60232,698,85860,60246,702,85860,60228,698


In [26]:
ak_hemibrain_traced_roi_connections_gb = ak_hemibrain_traced_roi_connections.groupby(["bodyId_pre", "bodyId_post"])
ak_hemibrain_traced_roi_connections_sorted = ak_hemibrain_traced_roi_connections[ak_hemibrain_traced_roi_connections_gb.permutation[ak_hemibrain_traced_roi_connections_gb.segments]]
ak_hemibrain_traced_roi_connections_sorted

,bodyId_pre,bodyId_post,roi,weight
0,200326126,264083994,SLP(R),3
1,200326126,295816140,ICL(R),2
4,200326126,296203440,SCL(R),1
5,200326126,325122109,SLP(R),2
6,200326126,326474963,SCL(R),1
...,...,...,...,...
4259619,7112622236,328283521,SMP(R),1
4259620,7112622236,357932060,SMP(R),1
4259621,7112622236,357940977,SMP(R),1
4259622,7112622236,358631450,SMP(R),1


In [31]:
ak_celegans_sorted.columns
ak_celegans_sorted



,continNum,EMseries,type,sec-tions,part-nerNum-ber,src,dst
175,548,JSE,chemical,1,2,234,341
1436,524,N2U,chemical,3,2,413,199
1001,2248,JSE,chemical,1,2,441,212
8160,5392,N2U,chemical,1,2,275,128
2845,1351,N2U,chemical,1,2,145,299
...,...,...,...,...,...,...,...
1438,525,N2U,chemical,1,2,319,273
10497,8018,N2U,chemical,1,2,328,270
4405,2432,N2U,chemical,1,2,147,128
5236,3095,N2U,chemical,1,2,84,396


In [20]:

ak_celegans_sorted['src'] = ak_celegans_sorted['pre']
del ak_celegans_sorted['pre']  # Remove the original column

ak_celegans_sorted['dst'] = ak_celegans_sorted['post']
del ak_celegans_sorted['post']  # Remove the original column

ak_celegans_sorted.columns


['continNum', 'EMseries', 'type', 'sec-tions', 'part-nerNum-ber', 'src', 'dst']

In [21]:
ak_microns_sorted.columns


['id',
 'pre_root_id',
 'post_root_id',
 'cleft_vx',
 'spine_vol_um3',
 'ctr_pt_x_nm',
 'ctr_pt_y_nm',
 'ctr_pt_z_nm',
 'pre_pos_x_vx',
 'pre_pos_y_vx',
 'pre_pos_z_vx',
 'ctr_pos_x_vx',
 'ctr_pos_y_vx',
 'ctr_pos_z_vx',
 'post_pos_x_vx',
 'post_pos_y_vx',
 'post_pos_z_vx']

In [22]:
ak_microns_sorted['src'] = ak_microns_sorted['pre_root_id']
del ak_microns_sorted['pre_root_id']  # Remove the original column

ak_microns_sorted['dst'] = ak_microns_sorted['post_root_id']
del ak_microns_sorted['post_root_id']  # Remove the original column

ak_microns_sorted.columns

['id',
 'cleft_vx',
 'spine_vol_um3',
 'ctr_pt_x_nm',
 'ctr_pt_y_nm',
 'ctr_pt_z_nm',
 'pre_pos_x_vx',
 'pre_pos_y_vx',
 'pre_pos_z_vx',
 'ctr_pos_x_vx',
 'ctr_pos_y_vx',
 'ctr_pos_z_vx',
 'post_pos_x_vx',
 'post_pos_y_vx',
 'post_pos_z_vx',
 'src',
 'dst']

In [27]:
ak_hemibrain_traced_roi_connections_sorted['src'] = ak_hemibrain_traced_roi_connections_sorted['bodyId_pre']
del ak_hemibrain_traced_roi_connections_sorted['bodyId_pre']  # Remove the original column

ak_hemibrain_traced_roi_connections_sorted['dst'] = ak_hemibrain_traced_roi_connections_sorted['bodyId_post']
del ak_hemibrain_traced_roi_connections_sorted['bodyId_post']  # Remove the original column

ak_hemibrain_traced_roi_connections_sorted.columns
ak_hemibrain_traced_roi_connections_sorted

,roi,weight,src,dst
0,SLP(R),3,200326126,264083994
1,ICL(R),2,200326126,295816140
4,SCL(R),1,200326126,296203440
5,SLP(R),2,200326126,325122109
6,SCL(R),1,200326126,326474963
...,...,...,...,...
4259619,SMP(R),1,7112622236,328283521
4259620,SMP(R),1,7112622236,357932060
4259621,SMP(R),1,7112622236,357940977
4259622,SMP(R),1,7112622236,358631450


In [ ]:
ak_celegans_sorted

In [ ]:
ar_celegans = ar.PropGraph()

ar_celegans.load_edge_attributes(ak_celegans_sorted, source_column="src", destination_column="dst", relationship_columns=["type"])

all_nodes = ak.concatenate([ak_celegans_sorted['src'], ak_celegans_sorted['dst']])
unique_nodes = ak.unique(all_nodes)
#unique_nodes.size
lbls = ak.array(["1"]*unique_nodes.size)
celegan_node_df = ak.DataFrame({"nodes": unique_nodes, "lbls":lbls})

ar_celegans.load_node_attributes(celegan_node_df,node_column="nodes", label_columns=["lbls"])


In [33]:
ak_hemibrain_traced_roi_connections_sorted["roi"] == "SLP(R)"

ar_hemibrain = ar.PropGraph()
ar_hemibrain.load_edge_attributes(ak_hemibrain_traced_roi_connections_sorted, source_column="src", destination_column="dst", relationship_columns=["roi"])

all_nodes = ak.concatenate([ak_hemibrain_traced_roi_connections_sorted['src'], ak_hemibrain_traced_roi_connections_sorted['dst']])
unique_nodes = ak.unique(all_nodes)
#unique_nodes.size
lbls = ak.array(["1"]*unique_nodes.size)
hemibrain_node_df = ak.DataFrame({"nodes": unique_nodes, "lbls":lbls})

ar_hemibrain.load_node_attributes(hemibrain_node_df,node_column="nodes", label_columns=["lbls"])


In [ ]:
subgraph = ar.PropGraph()
src_subgraph = ak.array([0, 1])
dst_subgraph = ak.array([1, 2])
lbls_subgraph = ak.array(["1"]*3)
rels_subgraph = ak.array([  "chemical", "chemical"])
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph, "rels":rels_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.arange(0,3), "lbls":lbls_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst",
                                relationship_columns=["rels"])
subgraph.load_node_attributes(node_df_h, node_column="nodes", label_columns=["lbls"])


In [ ]:
prop_graph= ar_celegans
# Grab vertex and edge data from the Arachne dataframes.
graph_node_information = prop_graph.get_node_attributes()
graph_edge_information = prop_graph.get_edge_attributes()
subgraph_node_information = subgraph.get_node_attributes()
subgraph_edge_information = subgraph.get_edge_attributes()

print(graph_node_information)
print(graph_edge_information)

print(subgraph_node_information)
print(subgraph_edge_information)



# Convert Arkouda dataframes to Pandas dataframes to NetworkX graph attributes.
G = nx.from_pandas_edgelist(graph_edge_information.to_pandas(), source='src',
                            target='dst', edge_attr=True, create_using=nx.DiGraph)
H = nx.from_pandas_edgelist(subgraph_edge_information.to_pandas(), source='src',
                                target='dst', edge_attr=True, create_using=nx.DiGraph)

# Convert graph node attributes to Pandas dfs, remove nodes, and convert rows to dicts.
graph_node_attributes = graph_node_information.to_pandas()
graph_nodes_from_df = list(graph_node_attributes.pop("nodes"))
graph_node_attributes = graph_node_attributes.to_dict('index')
graph_node_attributes_final = {}

# Convert subgraph node attributes to Pandas dfs remove nodes, and convert rows to dicts.
subgraph_node_attributes = subgraph_node_information.to_pandas()
subgraph_nodes_from_df = list(subgraph_node_attributes.pop("nodes"))
subgraph_node_attributes = subgraph_node_attributes.to_dict('index')
subgraph_node_attributes_final = {}


    # Convert Pandas index to original node index.
for key in graph_node_attributes:
    graph_node_attributes_final[graph_nodes_from_df[key]] = graph_node_attributes[key]

for key in subgraph_node_attributes:
        subgraph_node_attributes_final[subgraph_nodes_from_df[key]] = subgraph_node_attributes[key]


print("graph_node_attributes_final:")
print(graph_node_attributes_final)
print("subgraph_node_attributes_final:")
print(subgraph_node_attributes_final)
    # Set the node attributes for G and H from dicts.
nx.set_node_attributes(G, graph_node_attributes_final)
nx.set_node_attributes(H, subgraph_node_attributes_final)


In [ ]:

print(" Arachne....")
start = time.time()
isos = ar.subgraph_isomorphism(ar_celegans, subgraph)
end = time.time()
print(f"Finding {len(isos)/3} monomorphisms took {end-start} secs")
print("************************************************************")
print(" NetworkX... ")

    # Find subgraph isomorphisms of H in G.
start_time = time.time()
GM = nx.algorithms.isomorphism.DiGraphMatcher(G, H)
subgraph_isomorphisms = list(GM.subgraph_monomorphisms_iter())
elapsed_time = time.time() - start_time
print(f"NetworkX execution time: {elapsed_time} seconds")
print(f"NetworkX found: {len(subgraph_isomorphisms)} isos")
print("************************************************************")
print(" DotMotif....")
E = GrandIsoExecutor(graph=G)

# Create the search engine.
motif = Motif("""
A -> B 
B -> C
""")
start = time.time()

results = E.find(motif)
elapsed_time = time.time() - start_time
print(f"DotMotif execution time: {elapsed_time} seconds")
print(f"Dotmotif found: {len(subgraph_isomorphisms)} isos")
print(len(results))


In [ ]:
print("************************************************************")
print(" NetworkX... ")

    # Find subgraph isomorphisms of H in G.
start_time = time.time()
GM = nx.algorithms.isomorphism.DiGraphMatcher(G, H)
subgraph_isomorphisms = list(GM.subgraph_monomorphisms_iter())
elapsed_time = time.time() - start_time
print(f"NetworkX execution time: {elapsed_time} seconds")
print(f"NetworkX found: {len(subgraph_isomorphisms)} isos")
print("************************************************************")

In [ ]:
# Create a new graph from scratch:
#host = nx.fast_gnp_random_graph(100, 0.1, directed=True)

In [ ]:
ar_microns = ar.PropGraph()
ar_hemibrain = ar.PropGraph()

ar_microns.load_edge_attributes(ak_microns_sorted, source_column="src", destination_column="dst", relationship_columns=["id"])
ar_hemibrain.load_edge_attributes(ak_hemibrain_traced_roi_connections_sorted, source_column="src", destination_column="dst", relationship_columns=["roi"])

In [ ]:
print(f"C.elegans has {len(ar_celegans)} nodes and {ar_celegans.size()} edges")
print(f"MICrONS has {len(ar_microns)} nodes and {ar_microns.size()} edges")
print(f"Hemibrain has {len(ar_hemibrain)} nodes and {ar_hemibrain.size()} edges")

In [ ]:
#ak_celegans_sorted.to_hdf("/scratch/users/oaa9/experimentation/data/connectome/c.elegans/celegans_preprocessed")

In [ ]:
#ak_microns_sorted.to_hdf("/scratch/users/oaa9/experimentation/data/connectome/MICrONS/microns_preprocessed")

In [ ]:
#ak_celegans_sorted.to_hdf("/scratch/users/oaa9/experimentation/data/connectome/hemibrain/hemibrain_preprocessed")

In [ ]:
import time

In [ ]:
subgraph = ar.PropGraph()
src_subgraph = ak.array([2, 1, 0])
dst_subgraph = ak.array([1, 0, 2])
lbls_subgraph = ak.array(["1"]*3)
rels_subgraph = ak.array(["1", "1", "1"])
edge_df_h = ak.DataFrame({"src":src_subgraph, "dst":dst_subgraph, "rels":rels_subgraph})
node_df_h = ak.DataFrame({"nodes": ak.arange(0,3), "lbls":lbls_subgraph})
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst",
                                relationship_columns=["rels"])
subgraph.load_node_attributes(node_df_h, node_column="nodes", label_columns=["lbls"])
print("Calling Arachne....")
start = time.time()
isos = ar.subgraph_isomorphism(ar_celegans, subgraph)
end = time.time()
print(f"Finding {len(isos)/3} monomorphisms took {end-start} secs")